In [24]:
%pylab inline
import pandas as pd
import sklearn as skl
import scipy
import math
import numpy as np
from collections import defaultdict
import torch
from transformers import BertModel, BertTokenizerFast
from sklearn.metrics.pairwise import cosine_similarity

model_name = 'ai-forever/ruBert-base'
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).to("cuda")
document = pd.read_excel("../News_SGU_31077_Processed_1.xlsx")
texts = document["News_Text"]

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


AssertionError: Torch not compiled with CUDA enabled

In [16]:
print(texts)

0        9 мая состоялась общенародная акция «Бессмертн...
1        Председатель Государственной Думы Федерального...
2        Заместитель председателя Совета Безопасности Р...
3        Уважаемые ветераны университета, коллеги, студ...
4        Саратовский национальный исследовательский гос...
                               ...                        
31072    Международная база данных Scopus обнародовала ...
31073    Результаты научных исследований заведующего ка...
31074    Творческая редакция студенческого издания СГУ ...
31075    Председатель Правительства Российской Федераци...
31076    20 декабря 2012 года в диссертационном совете ...
Name: News_Text, Length: 31077, dtype: object


In [22]:
from tqdm import tqdm
word_embedding = defaultdict(list)

for text in tqdm(texts, desc="get embeddings"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # outputs.last_hidden_state.shape = [1, seq_len, 768]
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    embeddings = outputs.last_hidden_state[0].numpy()

    for token, embedding in zip(tokens, embeddings):
        if token.startswith("##"):  # Части слова (BPE-токенизация)
            continue
        word_embedding[token].append(embedding)

# Усреднение эмбеддингов для каждого слова
for word in tqdm(word_embedding, desc="mean embeddings"):
    word_embedding[word] = np.mean(word_embedding[word], axis=0)

get embeddings:   0%|          | 49/31077 [00:23<4:12:11,  2.05it/s]


KeyboardInterrupt: 

In [ ]:
print(embeddings)

In [5]:
from itertools import chain
all_words = list(set(chain.from_iterable(texts)))
print(len(all_words))

71192


In [6]:
# Словарь {слово: вектор}
word_embeddings = dict(zip(all_words, embeddings))

In [11]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import pandas as pd

def get_synonyms_optimized(target_word, word_vectors, all_vectors, words_list, top_k=10, min_similarity=0.7):
    """
    Оптимизированная функция поиска синонимов с предварительно вычисленными векторами
    
    Параметры:
        target_word: искомое слово
        word_vectors: словарь {слово: вектор}
        all_vectors: матрица всех векторов (n_words x dim)
        words_list: список всех слов в том же порядке, что и all_vectors
        top_k: количество возвращаемых синонимов
        min_similarity: минимальный порог схожести
    """
    if target_word not in word_vectors:
        return []
    
    # Получаем индекс целевого слова
    target_idx = words_list.index(target_word)
    target_vec = all_vectors[target_idx].reshape(1, -1)
    
    # Вычисляем схожести со всеми словами
    sim_scores = cosine_similarity(target_vec, all_vectors)[0]
    
    # Исключаем само слово
    sim_scores[target_idx] = -1
    
    # Находим индексы top_k наиболее похожих слов
    top_indices = np.argpartition(-sim_scores, top_k)[:top_k]
    
    # Фильтруем по минимальной схожести и создаем результат
    result = []
    for idx in top_indices:
        if sim_scores[idx] >= min_similarity:
            result.append((words_list[idx], sim_scores[idx]))
    
    # Сортируем по убыванию схожести
    return sorted(result, key=lambda x: -x[1])

# Предварительная подготовка данных
words_list = list(word_embeddings.keys())
all_vectors = np.array([word_embeddings[word] for word in words_list])

# Обработка с прогресс-баром
results = []
for word in tqdm(words_list, desc="Поиск синонимов"):
    similar_words = get_synonyms_optimized(word, word_embeddings, all_vectors, words_list)
    results.append({"Word": word, "Synonyms": similar_words})

# Создание DataFrame и сохранение
df_results = pd.DataFrame(results)
df_results.to_csv("optimized_similar_words.csv", index=False)

Поиск синонимов: 100%|██████████| 31077/31077 [53:38<00:00,  9.65it/s]  
